# Initiliation

In [1]:
!pip install transformers datasets --quiet

In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/URL'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/URL


In [5]:
dfPJ = pd.read_csv('myPers_CON_URL.csv', sep=",", error_bad_lines=False)
dfPJ

,text,label
0,likes the sound of thunder,0
1,is so sleepy its not even funny thats ...,0
2,is sore and wants the knot of muscles ...,0
3,likes how the day sounds in this new s...,0
4,is home 3,0
...,...,...
9912,little things give you away,0
9913,is wishing it was saturday,1
9914,is studying hard for the gre,1
9915,snipers get more head,0


In [6]:
df_clean= []
for i,row in dfPJ.iterrows():
  df_clean.append({
      'text': str(row['text']),
      'label': int(row['label']),
  })

dfPJ = pd.DataFrame(df_clean)
dfPJ

,text,label
0,likes the sound of thunder,0
1,is so sleepy its not even funny thats ...,0
2,is sore and wants the knot of muscles ...,0
3,likes how the day sounds in this new s...,0
4,is home 3,0
...,...,...
9912,little things give you away,0
9913,is wishing it was saturday,1
9914,is studying hard for the gre,1
9915,snipers get more head,0


# Model Training

In [7]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [8]:
train, test = train_test_split(dfPJ, test_size=0.2, random_state=0, stratify=dfPJ.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_CON", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [9]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 4.4 MB/s 
     |████████████████████████████████| 209 kB 48.0 MB/s 
     |████████████████████████████████| 80 kB 8.2 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 149 kB 38.0 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 112 kB 48.2 MB/s 


In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [11]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [12]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_myPers_CON"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-24 13:54:33,065] A new study created in RDB with name: SIMCSE_BERT_myPers_CON


A new study created in RDB with name: SIMCSE_BERT_myPers_CON


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weigh

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.689500,0.676174,0.577117,0.532808,0.577397,0.555676
2,0.662200,0.669825,0.582157,0.564175,0.577020,0.569368
3,0.633700,0.672615,0.587702,0.566374,0.583917,0.573416
4,0.617500,0.672099,0.587198,0.576935,0.582154,0.578336
4,0.609800,0.672040,0.585685,0.575590,0.580570,0.576938


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.59      0.82      0.68      1073
           1       0.60      0.31      0.41       911

    accuracy                           0.59      1984
   macro avg       0.59      0.57      0.55      1984
weighted avg       0.59      0.59      0.56      1984

[[885 188]
 [627 284]]


[I 2021-12-25 05:28:58,803] Trial 49 pruned. 


Trial 49 pruned. 


[I 2021-12-25 05:28:58,853] A new study created in memory with name: no-name-92bef35c-4b3e-4714-8b2c-6342a31faec0


A new study created in memory with name: no-name-92bef35c-4b3e-4714-8b2c-6342a31faec0


In [13]:
storage_name

'sqlite:///SIMCSE_BERT_myPers_CON.db'

In [14]:
study_name

'SIMCSE_BERT_myPers_CON'

In [15]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-25 05:28:58,994] Using an existing study with name 'SIMCSE_BERT_myPers_CON' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_myPers_CON' instead of creating a new one.


In [16]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.575590,3.001811e-06,4.105974,8,7.674193e-12,COMPLETE
1,1,0.476413,4.051154e-07,4.350658,2,5.144719e-03,COMPLETE
2,2,0.350998,9.297905e-05,6.435028,8,3.103698e-06,COMPLETE
3,3,0.584742,1.189616e-05,1.769990,2,7.353256e-05,COMPLETE
4,4,0.350998,1.497365e-03,3.599022,8,3.895097e-10,COMPLETE
5,5,0.314680,7.779768e-02,1.081947,2,4.260396e-08,PRUNED
6,6,0.373820,1.214030e-04,3.594346,16,3.132025e-06,PRUNED
7,7,0.588056,4.169727e-05,1.113719,4,1.303774e-04,COMPLETE
8,8,0.609633,1.303811e-05,4.981086,8,2.694548e-04,COMPLETE
9,9,0.437731,8.831596e-08,5.703767,4,1.025546e-06,PRUNED


In [17]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [18]:
best_run

BestRun(run_id='17', objective=0.6181159420289855, hyperparameters={'learning_rate': 1.7443883599512133e-05, 'num_train_epochs': 5.009529088071898, 'per_device_train_batch_size': 16, 'weight_decay': 2.4793361471561817e-07})

In [19]:
optuna.visualization.plot_intermediate_values(study)

In [20]:
optuna.visualization.plot_parallel_coordinate(study)

In [21]:
optuna.visualization.plot_optimization_history(study)

In [22]:
optuna.visualization.plot_contour(study)

In [23]:
optuna.visualization.plot_slice(study)

In [24]:
optuna.visualization.plot_edf(study)